In [24]:
%pylab inline
import pandas as pd
import sklearn.ensemble as sk_ensemble
import sklearn.cross_validation as sk_cv

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [21]:
def rmsle(actual, predicted):
    error = np.log1p(predicted) - np.log1p(actual)
    return np.sqrt(np.mean(np.square(error)))

In [2]:
train_df = pd.read_csv('../train_set_adjusted.csv')

In [3]:
print train_df.shape
print train_df.columns

(30213, 66)
Index([u'tube_assembly_id', u'quote_date', u'annual_usage',
       u'min_order_quantity', u'quantity', u'cost', u'year', u'quantity_rep',
       u'supplier_S-0003', u'supplier_S-0004', u'supplier_S-0005',
       u'supplier_S-0006', u'supplier_S-0007', u'supplier_S-0008',
       u'supplier_S-0009', u'supplier_S-0011', u'supplier_S-0012',
       u'supplier_S-0013', u'supplier_S-0014', u'supplier_S-0015',
       u'supplier_S-0018', u'supplier_S-0022', u'supplier_S-0023',
       u'supplier_S-0024', u'supplier_S-0025', u'supplier_S-0026',
       u'supplier_S-0027', u'supplier_S-0029', u'supplier_S-0030',
       u'supplier_S-0031', u'supplier_S-0041', u'supplier_S-0042',
       u'supplier_S-0043', u'supplier_S-0046', u'supplier_S-0050',
       u'supplier_S-0051', u'supplier_S-0054', u'supplier_S-0056',
       u'supplier_S-0058', u'supplier_S-0059', u'supplier_S-0060',
       u'supplier_S-0061', u'supplier_S-0062', u'supplier_S-0064',
       u'supplier_S-0066', u'supplier_S-0068',

In [4]:
tube = pd.read_csv('../tube_material_id_imputed_dummies_drop_ns.csv')

In [5]:
print tube.shape
print tube.columns

(21198, 84)
Index([u'tube_assembly_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'num_boss', u'num_bracket', u'other',
       u'material_id_SP-0008', u'material_id_SP-0019', u'material_id_SP-0028',
       u'material_id_SP-0029', u'material_id_SP-0030', u'material_id_SP-0031',
       u'material_id_SP-0032', u'material_id_SP-0033', u'material_id_SP-0034',
       u'material_id_SP-0035', u'material_id_SP-0036', u'material_id_SP-0037',
       u'material_id_SP-0038', u'material_id_SP-0039', u'material_id_SP-0041',
       u'material_id_SP-0044', u'material_id_SP-0045', u'material_id_SP-0046',
       u'material_id_SP-0048', u'end_a_1x_Y', u'end_a_2x_Y', u'end_x_1x_Y',
       u'end_x_2x_Y', u'end_a_EF-001', u'end_a_EF-002', u'end_a_EF-003',
       u'end_a_EF-004', u'end_a_EF-005', u'end_a_EF-006', u'end_a_EF-007',
       u'end_a_EF-008', u'end_a_EF-009', u'end_a_EF-010', u'end_a_EF-011',
       u'end_a_EF-012', u'end_a_EF-013', u'end_a_EF-014', u'end_a_EF-015',
   

In [6]:
spec = pd.read_csv('../spec_dummies.csv')

In [7]:
print spec.shape
print spec.columns

(21198, 86)
Index([u'tube_assembly_id', u'SP-0001', u'SP-0002', u'SP-0003', u'SP-0004',
       u'SP-0005', u'SP-0006', u'SP-0007', u'SP-0009', u'SP-0010', u'SP-0011',
       u'SP-0012', u'SP-0013', u'SP-0014', u'SP-0015', u'SP-0016', u'SP-0017',
       u'SP-0018', u'SP-0019', u'SP-0020', u'SP-0021', u'SP-0022', u'SP-0023',
       u'SP-0024', u'SP-0025', u'SP-0026', u'SP-0027', u'SP-0028', u'SP-0029',
       u'SP-0030', u'SP-0033', u'SP-0035', u'SP-0036', u'SP-0037', u'SP-0038',
       u'SP-0039', u'SP-0040', u'SP-0042', u'SP-0043', u'SP-0044', u'SP-0046',
       u'SP-0047', u'SP-0049', u'SP-0050', u'SP-0051', u'SP-0052', u'SP-0053',
       u'SP-0054', u'SP-0055', u'SP-0056', u'SP-0057', u'SP-0058', u'SP-0059',
       u'SP-0060', u'SP-0061', u'SP-0062', u'SP-0063', u'SP-0064', u'SP-0065',
       u'SP-0066', u'SP-0067', u'SP-0068', u'SP-0069', u'SP-0070', u'SP-0071',
       u'SP-0072', u'SP-0073', u'SP-0074', u'SP-0075', u'SP-0076', u'SP-0077',
       u'SP-0078', u'SP-0079', u'SP-0080', 

In [8]:
comp = pd.read_csv('../comp_type_dummies.csv')

In [9]:
print comp.shape
print comp.columns

(21198, 30)
Index([u'tube_assembly_id', u'CP-001', u'CP-002', u'CP-003', u'CP-004',
       u'CP-005', u'CP-006', u'CP-007', u'CP-008', u'CP-009', u'CP-010',
       u'CP-011', u'CP-012', u'CP-014', u'CP-015', u'CP-016', u'CP-017',
       u'CP-018', u'CP-019', u'CP-020', u'CP-021', u'CP-022', u'CP-023',
       u'CP-024', u'CP-025', u'CP-026', u'CP-027', u'CP-028', u'CP-029',
       u'OTHER'],
      dtype='object')


In [10]:
comp_type_weight = pd.read_csv('../comp_type_weight.csv')

In [11]:
print comp_type_weight.shape
print comp_type_weight.columns

(21198, 30)
Index([u'tube_assembly_id', u'CP-001', u'CP-002', u'CP-003', u'CP-004',
       u'CP-005', u'CP-006', u'CP-007', u'CP-008', u'CP-009', u'CP-010',
       u'CP-011', u'CP-012', u'CP-014', u'CP-015', u'CP-016', u'CP-017',
       u'CP-018', u'CP-019', u'CP-020', u'CP-021', u'CP-022', u'CP-023',
       u'CP-024', u'CP-025', u'CP-026', u'CP-027', u'CP-028', u'CP-029',
       u'OTHER'],
      dtype='object')


In [12]:
tube_vol = pd.read_csv('../tube_volume.csv')

In [14]:
train = pd.merge(train_df, tube)
train = pd.merge(train, spec)
train = pd.merge(train, comp_type_weight)
train = pd.merge(train, tube_vol)

In [15]:
train.dtypes

tube_assembly_id       object
quote_date             object
annual_usage            int64
min_order_quantity      int64
quantity                int64
cost                  float64
year                    int64
quantity_rep          float64
supplier_S-0003       float64
supplier_S-0004       float64
supplier_S-0005       float64
supplier_S-0006       float64
supplier_S-0007       float64
supplier_S-0008       float64
supplier_S-0009       float64
supplier_S-0011       float64
supplier_S-0012       float64
supplier_S-0013       float64
supplier_S-0014       float64
supplier_S-0015       float64
supplier_S-0018       float64
supplier_S-0022       float64
supplier_S-0023       float64
supplier_S-0024       float64
supplier_S-0025       float64
supplier_S-0026       float64
supplier_S-0027       float64
supplier_S-0029       float64
supplier_S-0030       float64
supplier_S-0031       float64
                       ...   
CP-001                float64
CP-002                float64
CP-003    

In [31]:
train_sub_train, train_sub_cv = sk_cv.train_test_split(train, train_size = 0.7, random_state = 0)

In [32]:
print train_sub_train.shape
print train_sub_cv.shape

(21149, 264)
(9064, 264)


In [43]:
X = train_sub_train.drop(['tube_assembly_id', 'quote_date', 'cost'], axis=1).values
Y = train_sub_train.cost
rf = sk_ensemble.RandomForestRegressor(n_estimators=100, verbose=0, random_state=0, n_jobs=4, oob_score=True)
rf = rf.fit(X, Y)

In [44]:
X_cv = train_sub_cv.drop(['tube_assembly_id', 'quote_date', 'cost'], axis=1).values
y_cv = train_sub_cv.cost
y_cv_fitted = rf.predict(X_cv)

In [45]:
rmsle(y_cv, y_cv_fitted)

0.24964471987586298